In [ ]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
pip install keras-tuner seaborn scikit-learn tensorflow


In [ ]:
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf


In [ ]:
df = sns.load_dataset('iris')
df.head()
x = df.drop(columns=['sepal_length'])
y = df['sepal_length']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
le = LabelEncoder()
x_train['species'] = le.fit_transform(x_train['species'])
x_test['species']= le.transform(x_test['species'])


def test_model(hp):
    model=Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Dense(hp.Int('unit1',min_value=32,max_value=128,step=32),activation=hp.Choice('activation1',['relu','tanh','sigmoid'])))
    model.add(Dense(hp.Int('unit2',min_value=32,max_value=128,step=32),activation=hp.Choice('activation2',['relu','tanh','sigmoid'])))
    model.add(Dense(1))
    model.compile(optimizer=SGD(hp.Choice('optimize',[0.00001,0.0001,0.001]),momentum=hp.Choice('momentum',[0.0,0.5,0.9])),loss='mean_squared_error',metrics=['mae'])
    return model



In [ ]:
tuner = kt.GridSearch(
    test_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=1,
    directory='test',
    project_name='iris_reg2'
)


tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, mae = best_model.evaluate(x_test, y_test)
print(f"Test MAE: {mae}")


Trial 10 Complete [00h 00m 03s]
val_mae: 6.752825736999512

Best val_mae So Far: 0.24899229407310486
Total elapsed time: 00h 00m 33s
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 36.3050 - mae: 5.9594 - val_loss: 19.9201 - val_mae: 4.4349
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 13.6244 - mae: 3.5363 - val_loss: 0.9993 - val_mae: 0.8861
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 4.6342 - mae: 1.6736 - val_loss: 2.6289 - val_mae: 1.3354
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 2.5289 - mae: 1.3123 - val_loss: 6.0576 - val_mae: 2.4372
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 5.0775 - mae: 2.2028 - val_loss: 1.0255 - val_mae: 0.9758
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.7351 - mae: 0.7311 - val_loss: 2.2823 - val_mae: 1.4008
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.0982 - mae: 1.2767 - val_loss: 0.2149 - val_mae: 0.3820
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.